In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, Xception, ResNet50V2, EfficientNetB2, DenseNet201, InceptionV3, ConvNeXtTiny
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np


In [2]:
AUTOTUNE = tf.data.AUTOTUNE

# Function to preprocess images
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

# Load datasets using tf.data pipeline
train_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/train',
    image_size=(224, 224),
    batch_size=32
).map(preprocess_image).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/val',
    image_size=(224, 224),
    batch_size=32
).map(preprocess_image).cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    'dataset/test',
    image_size=(224, 224),
    batch_size=32
).map(preprocess_image).cache().prefetch(buffer_size=AUTOTUNE)
    
    


Found 3919 files belonging to 4 classes.
Found 395 files belonging to 4 classes.
Found 379 files belonging to 4 classes.


In [3]:


def trainModel(base_model, label):

    base_model.trainable = False  # Freeze the base model initially
    
    # Add task-specific layers with improved Dropout
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3), 
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5), 
        layers.Dense(4, activation='softmax')  # Adjust number of classes accordingly
    ])
    
    # Implement Learning Rate Decay
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=1000,
        decay_rate=0.96,
        staircase=True
    )
    
    # Compile the model with improved learning rate tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='sparse_categorical_crossentropy',  
        metrics=['accuracy']
    )
    
    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    checkpoint_callback = ModelCheckpoint(label + "_best.keras", save_best_only=True, monitor="val_loss", verbose=1)
    
    # Train the model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=15,
        callbacks=[early_stopping, checkpoint_callback],
        verbose=1
    )
    
    # Fine-tune the entire model
    base_model.trainable = True  # Unfreeze the base model
    
    # Recompile model with a lower learning rate for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='sparse_categorical_crossentropy',  # Or categorical if using one-hot
        metrics=['accuracy']
    )
    
    # Fine-tune the model with a new EarlyStopping instance
    history_fine = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=15,
        callbacks=[early_stopping, checkpoint_callback],  # New callbacks
        verbose=1
    )
    
   
    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_ds, verbose=1)

    print(f"Model: {label}")
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

In [ ]:
#https://keras.io/api/applications/
# Load Pretrained Models, train and test them

efficientNetB2_model = EfficientNetB2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(efficientNetB2_model, "EfficientNetB2")

resNet50V2_model = ResNet50V2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(resNet50V2_model, "ResNet50V2")

xception_model = Xception(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(xception_model, "Xception")

mobileNetV2_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(mobileNetV2_model, "MobileNetV2")

efficientNetB0_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(mobileNetV2_model, "EfficientNetB0")

denseNet201_model = DenseNet201(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(denseNet201_model, "DenseNet201")

inceptionV3_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(inceptionV3_model, "InceptionV3")

convNeXtTiny_model = ConvNeXtTiny(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
trainModel(convNeXtTiny_model, "ConvNeXtTiny")


Epoch 1/15
123/123 [==============================] - ETA: 0s - loss: 0.6601 - accuracy: 0.7425
Epoch 1: val_loss improved from inf to 0.38757, saving model to DenseNet201_best.keras
123/123 [==============================] - 460s 4s/step - loss: 0.6601 - accuracy: 0.7425 - val_loss: 0.3876 - val_accuracy: 0.8633
Epoch 2/15
123/123 [==============================] - ETA: 0s - loss: 0.4091 - accuracy: 0.8528
Epoch 2: val_loss improved from 0.38757 to 0.36738, saving model to DenseNet201_best.keras
123/123 [==============================] - 450s 4s/step - loss: 0.4091 - accuracy: 0.8528 - val_loss: 0.3674 - val_accuracy: 0.8861
Epoch 3/15
123/123 [==============================] - ETA: 0s - loss: 0.3753 - accuracy: 0.8637
Epoch 3: val_loss improved from 0.36738 to 0.36225, saving model to DenseNet201_best.keras
123/123 [==============================] - 417s 3s/step - loss: 0.3753 - accuracy: 0.8637 - val_loss: 0.3623 - val_accuracy: 0.8861
Epoch 4/15
123/123 [===========================

In [ ]:
Model: ResNet50V2
Test Loss: 0.2777522802352905
Test Accuracy: 0.9287598729133606

Model: Xception
Test Loss: 0.18646970391273499
Test Accuracy: 0.9472295641899109

Model: MobileNetV2
Test Loss: 1.1013202667236328
Test Accuracy: 0.831134557723999

Model: EfficientNetB0
Test Loss: 1.2719537019729614
Test Accuracy: 0.8205804824829102
